In [9]:
import pandas as pd
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool, CustomJS
from bokeh.models.widgets import Select
from bokeh.io import vform

# Open the excel file as an object
# xlsfile = pd.ExcelFile('data/top100.xlsx', header=None)
# dframe = xlsfile.parse(header=None)
# dframe = dframe.rename(columns={30: 'Datetime'})

# Ускоряем загрузку скрипта с помощью преварительно сохранённого в файл DataFrame
# Сохранять с помощью dframe.to_pickle('data/dframe')
dframe = pd.read_pickle('data/dframe')

In [10]:
players = dframe[1].unique()
len(players)

64

In [11]:
# Отбираем игры конкретного игрока, для примера
# rand_counter = randint(1, len(players))
rand_counter = 0
player_match = dframe[dframe[1] == players[rand_counter]]
nickname = dframe[0][rand_counter]
len(player_match)

556

In [12]:
# player_match.set_index('Datetime').groupby(pd.TimeGrouper('M'))[0].apply(lambda x: x.count())
# player_match.set_index('Datetime').groupby(pd.TimeGrouper('M'))[4].apply(lambda x: len(x.unique()))

# Получаем список героев по месяцам:
uniq_heroes_names_month = player_match.set_index('Datetime').groupby(pd.TimeGrouper('M'))[4].unique()
uniq_heroes_names_month

Datetime
2014-04-30    [Invoker, Meepo, Skywrath Mage, Faceless Void,...
2014-05-31    [Io, Tinker, Gyrocopter, Ursa, Slark, Ember Sp...
2014-06-30    [Broodmother, Invoker, Mirana, Clockwerk, Snip...
2014-07-31    [Broodmother, Earth Spirit, Nature's Prophet, ...
2014-08-31                                                   []
2014-09-30                                                 [Io]
2014-10-31                                                   []
2014-11-30                                                   []
2014-12-31    [Meepo, Faceless Void, Storm Spirit, Disruptor...
2015-01-31    [Anti-Mage, Troll Warlord, Juggernaut, Terrorb...
2015-02-28    [Earth Spirit, Juggernaut, Io, Naga Siren, Sni...
2015-03-31    [Io, Luna, Tiny, Juggernaut, Anti-Mage, Phanto...
2015-04-30    [Troll Warlord, Lifestealer, Drow Ranger, Husk...
2015-05-31                                [Ursa, Queen of Pain]
2015-06-30                                [Gyrocopter, Leshrac]
2015-07-31                     

In [13]:
# Получаем количество уникальных героев по месяцам: (объект pandas.core.series.Series)
uniq_heroes_month = player_match.set_index('Datetime').groupby(pd.TimeGrouper('M'))[4].apply(lambda x: len(x.unique()))
uniq_heroes_month

Datetime
2014-04-30     7
2014-05-31    36
2014-06-30    13
2014-07-31    18
2014-08-31     0
2014-09-30     1
2014-10-31     0
2014-11-30     0
2014-12-31    32
2015-01-31    38
2015-02-28    10
2015-03-31    10
2015-04-30    23
2015-05-31     2
2015-06-30     2
2015-07-31     3
2015-08-31     2
2015-09-30     2
2015-10-31     2
2015-11-30     2
2015-12-31     1
Name: 4, dtype: int64

In [14]:
# Рисуем графики:
output_file("legend.html", title="legend.py example")
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select, crosshair"

list_players = [dframe[dframe[1] == players[x]].set_index('Datetime').groupby(pd.TimeGrouper('M'))[4].apply(lambda x: len(x.unique())) for x in range(0, len(players))]
list_players_indexes = [list_players[x].keys() for x in range(0, len(list_players))]

list_uniq_heroes = [dframe[dframe[1] == players[x]].set_index('Datetime').groupby(pd.TimeGrouper('M'))[4].apply(lambda x: len(x.unique())) for x in range(0, len(players))]

source = ColumnDataSource(
        data=dict(
            x=uniq_heroes_month.index,
            y=uniq_heroes_month,
            players=list_players,
            dates=list_players_indexes,
            nick=[nickname for x in range(len(uniq_heroes_month.index))],
            desc=[uniq_heroes_month[x] for x in range(len(uniq_heroes_month.index))],
            time=[uniq_heroes_month.keys()[x].strftime('%m-%Y') for x in range(len(uniq_heroes_month.index))],
            dframe=dframe[0].unique(),
            list_uniq_heroes=list_uniq_heroes,
            since=[list_players[0].keys()[0].strftime('%m-%Y') for x in range(len(uniq_heroes_month.index))],
            first_games=[list_players[x].keys()[0].strftime('%m-%Y') for x in range(0, len(players))],
        )
    )

In [15]:
hover = HoverTool(
        tooltips=[
            ("Index", "$index"),
            ("Date", "@time"),
            ("Nick:", "@nick"),
            ("Unique heroes:", "@desc"),
            ("Playing since:", "@since"),
        ]
    )

In [16]:
# График с группировкой по месяцам
plot1 = figure(width=1000, height=600, name="foo", title="Unique heroes per month",
               x_axis_type="datetime", tools=[hover, TOOLS])
plot1.xaxis.axis_label = 'Date'
plot1.yaxis.axis_label = 'Heroes'


plot1.circle('x', 'y', size=10, source=source)
plot1.line('x', 'y', source=source)

# Сам график для одного игрока можно уже выводить, но дальше добавляется интерактивная часть
# show(plot1)

In [17]:
select_callback = CustomJS(args=dict(source=source), code="""
    var data = source.get('data');
    var f = cb_obj.get('value').split('\t', 1)
    x = data['x']
    y = data['y']

    nickname = new Array();
    datetime = new Array();
    fgames = new Array();
    for (i = 0; i < data['dates'][f].length; i++) {
            nickname.push(data['dframe'][f])
            datetime.push(new Date(data['dates'][f][i]))
            fgames.push(data['first_games'][f])
        }
    data['nick'] = nickname
    data['x'] = data['dates'][f]
    data['y'] = data['players'][f]
    data['desc'] = data['list_uniq_heroes'][f]
    var date = new Date(data['dates'][f][0]);
    data['time'] = datetime
    data['since'] = fgames
    source.trigger('change');
""")

select = Select(title="Player:", value="None", options=[str(x)+ '\t' + dframe[0].unique()[x] for x in range(0, len(players))], callback=select_callback)

layout = vform(select,  plot1)
output_file("legend.html", title="legend.py example")
show(layout)

ERROR:/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/bokeh/core/validation/check.py:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: 5fb22f1d-b5ad-40f7-90d1-6fe69b5ab2d5


ERROR:/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/bokeh/core/validation/check.py:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: 5fb22f1d-b5ad-40f7-90d1-6fe69b5ab2d5


ERROR:/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/bokeh/core/validation/check.py:W-1004 (BOTH_CHILD_AND_ROOT): Models should not be a document root if they are in a layout box: Figure, ViewModel:Plot, ref _id: 9fd20c1a-c16f-49e6-836a-c72f36c9a998
